In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## 1. github 클론

In [ ]:
! git clone https://github.com/hyomee2/sswu-lc5500-aiapp.git

Cloning into 'sswu-lc5500-aiapp'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 197 (delta 48), reused 74 (delta 37), pack-reused 107 (from 1)
Receiving objects: 100% (197/197), 337.36 KiB | 17.76 MiB/s, done.
Resolving deltas: 100% (50/50), done.


## 2. 데이터셋 불러와서 훈련 포맷으로 맞추기

In [ ]:
import pandas as pd
import json

df = pd.read_csv("/content/sswu-lc5500-aiapp/Llama/dataset/Llama_data.csv").dropna(subset=["Q", " A"]) # 결측치 제외 후 불러오기

# JSON 리스트 만들기
records = []
for _, row in df.iterrows():
    records.append({
        "instruction": row["Q"].strip(), # 질문 앞뒤 공백 제거
        "input": "", # input은 비움
        "output": str(row[" A"]).strip() # A 앞에 들어있는 공백 제거
    })

# JSON 저장
with open("train_data.json", "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=2)

## 3. Huggingface 데이터셋으로 변환 및 로딩

In [ ]:
import pandas as pd
from datasets import Dataset

# JSON을 pandas로 읽기
df = pd.read_json("train_data.json")

# Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)
# train/validation 95:5으로 나누기(90:10으로도 해볼까)
split_dataset = dataset.train_test_split(test_size=0.05)

train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]


## 4. HuggingFace 모델 불러오기

In [ ]:
!pip install transformers accelerate peft bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from huggingface_hub import login
from google.colab import userdata

hugging_face_token = userdata.get(HUGGING_FACE_TOKEN)
login(hugging_face_token)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model_name = "beomi/Llama-3-Open-Ko-8B"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # pad_token 설정
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    load_in_8bit=True, # LoRA를 위해 8bit 로딩
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

...

model-00005-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# LoRA를 위한 준비
model = prepare_model_for_kbit_training(model)

# LoRA 설정
lora_config = LoraConfig(
    r=8, # LoRA의 랭크(저차원 행렬의 크기). 보통 4~16 사이로 설정
    lora_alpha=32, # LoRA 내 스케일링 팩터. 학습 안정성에 도움
    target_modules=["q_proj", "v_proj"],  # LoRA가 어떤 신경망 내부 레이어(모듈)에만 적용될지 지정
    lora_dropout=0.05, # 과적합 방지를 위해 LoRA 적용 시 드롭아웃 확률
    bias="none", # bias 파라미터 업데이트 여부(none, all, lora 등)
    task_type="CAUSAL_LM" # 작업 유형 지정. 여기서는 언어 생성용 인과적 언어모델
)

model = get_peft_model(model, lora_config)


*** target_modules=["q_proj", "v_proj"]**

**Q, K, V란?**

Transformer 모델의 핵심인 어텐션(attention) 계산에서
Q: Query (질문)
K: Key (키)
V: Value (값)

이 세 개 행렬(벡터)을 만들어서 어텐션 점수를 계산.


**"q_proj", "v_proj"는 무엇인가?**

Transformer 구현체마다 다르지만, 보통 Q, K, V를 만들 때
q_proj, k_proj, v_proj라는 이름으로 각각의 선형 변환(Linear layer, 즉 행렬곱)을 만든다.

e.g.,
``` python
self.q_proj = nn.Linear(hidden_dim, hidden_dim)
self.k_proj = nn.Linear(hidden_dim, hidden_dim)
self.v_proj = nn.Linear(hidden_dim, hidden_dim)
```


**그럼 target_modules=["q_proj", "v_proj"]는?**

LoRA를 Q 프로젝션 레이어와 V 프로젝션 레이어에만 적용하겠다는 의미. 즉, Q와 V를 만드는 선형 변환에 LoRA 어댑터가 붙어서 학습된다. K는 빼고 Q, V에만 적용하는 건 실험적으로 성능이나 효율이 좋다고 알려진 경우가 많다. Q는 모델이 "무엇을 집중할지"를 결정하는 쿼리 정보고, V는 실제로 어텐션에서 참고하는 값이라, 이 두 부분을 조정하는 것이 모델 미세조정에 효과적일 때가 많다. K는 상대적으로 업데이트 효과가 적거나 중복될 수 있어서 제외하기도 한다.





## 5. 텍스트를 LLM 학습 포맷으로 변환

In [ ]:
# Prompt 템플릿 함수
def format_prompt(example):
    return f"""### 질문:\n{example['instruction']}\n\n### 답변:\n{example['output']}"""

**format_prompt(example)**

데이터를 LLM에 학습시키기 좋은 문자열 형태(Prompt)로 만들어준다.

e.g.,

```
### 질문:

12시 땡이다!

### 답변:

하루 또 가뿌네.
```

In [ ]:
# 토크나이징
def tokenize(example):
    prompt = format_prompt(example)
    return tokenizer(prompt, padding="max_length", truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(tokenize, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/11157 [00:00<?, ? examples/s]

Map:   0%|          | 0/588 [00:00<?, ? examples/s]

**tokenize()**

위에서 만든 문자열 prompt을 모델 학습용 숫자 토큰들로 변환한다.

padding="max_length": 길이가 512보다 짧으면 빈 자리를 채운다.

truncation=True: 너무 길면 512까지만 자름

결과는 다음과 같은 dict 형태

e.g.,
```python
{
  'input_ids': [...숫자들...],
  'attention_mask': [...1, 1, 1, 0, 0, 0...]  # pad된 부분은 0
}
```




## 6. 모델 학습을 위한 collator(데이터 배치 준비 도우미) 준비

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# min=False -> GPT,LLAMA 같은 casual language model 학습에 적합

## 7. Trainer 설정

**TrainingAruguments**

: 모델 학습과 관련된 다양한 하이퍼파라미터와 설정들을 모아놓은 객체. Trainer에게 어떻게 학습할지 알려주는 설정 모음집이라 할 수 있다.

In [ ]:
from transformers import TrainingArguments, Trainer

# Trainer 설정
training_args = TrainingArguments(
    output_dir="/content/gdrive/MyDrive/models/checkpoints", # 학습 체크포인트(모델 가중치 등)를 저장할 폴더 경로
    per_device_train_batch_size=2, # 학습 시 GPU 1대당 사용할 배치 사이즈
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=4, # 평가 시 GPU 1대당 사용할 배치 사이즈
    num_train_epochs=3, # 총 학습 Epoch 수(3번 데이터셋을 반복 학습)
    logging_dir="./logs", # 로그(학습 진행상황 등)를 저장할 디렉토리
    logging_steps=10, # 몇 스텝마다 로그를 기록할지(여기선 10스텝마다)
    save_strategy="steps", # 체크포인트 저장 주기(여기선 epoch마다 저장)
    #evaluation_strategy="epoch", # 평가 주기(여기선 epoch마다 평가)
    fp16=True, # 16-bit half precision 사용 여부(속도 향상 및 메모리 절약)
    save_steps=50,
    save_total_limit=2, # 저장할 체크포인트 최대 개수(초과 시 오래된 것부터 삭제)
)

trainer = Trainer(
    model=model, # 학습할 PyTorch 모델
    args=training_args, # TrainingArguments 객체(학습 설정)
    train_dataset=tokenized_train, # 학습용 데이터셋
    eval_dataset=tokenized_val, # 평가용 데이터셋
    tokenizer=tokenizer, # 토크나이저(토큰 ID 변환 및 디코딩 등에 사용)
    data_collator=data_collator, # 배치 데이터를 만들 떄 자동으로 padding 등을 처리하는 함수
)

<ipython-input-17-1f433fae398d>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
import torch

# torch.cuda.empty_cache()

# 학습 시작
# trainer.train(resume_from_checkpoint=True)
# wanb.init
trainer.train()

# 모델 저장
model.save_pretrained("./finetuned-kollama-dialect")
tokenizer.save_pretrained("./finetuned-kollama-dialect")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hyomee (hyomee-organization) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,3.185300
20,2.781300
30,2.479400
40,2.456200
50,2.410200
60,2.392200
70,2.214500
80,2.273000
90,2.203200
100,2.171000


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reen

Step,Training Loss
10,3.185300
20,2.781300
30,2.479400
40,2.456200
50,2.410200
60,2.392200
70,2.214500
80,2.273000
90,2.203200
100,2.171000


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reen